# Using mistralai/Mistral-7B-Instruct-v0.3 model

In [ ]:
!pip install "distilabel[hf-inference-endpoints]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 682.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found exis

In [ ]:
!pip install "transformers~=4.0" "torch~=2.0" "huggingface_hub~=0.24.0"

In [ ]:
from distilabel.llms import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import (
    LoadDataFromHub,
    GroupColumns,
    FormatTextGenerationDPO,
    PreferenceToArgilla,
)
from distilabel.steps.tasks import TextGeneration, UltraFeedback

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import login

login(token=os.getenv("HF_TOKEN") or userdata.get("HF_TOKEN"), add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from distilabel.pipeline import Pipeline
from distilabel.llms.huggingface import InferenceEndpointsLLM
from distilabel.steps.generators.data import LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration
from datasets import load_dataset


In [ ]:
DATA = [
        {
          "instruction": "Label the paragraph as either Claim, Evidence, Corroboration, or Contextualization. paragraph: Regular exercise improves cardiovascular health."
        },
        {
          "instruction": """Label the paragraph as either Claim, Evidence, Corroboration, or Contextualization.
          paragraph: A study published in the Journal
          of the American College of Cardiology in 2022 found that individuals who engaged in at least 150 minutes of moderate-intensity exercise
           per week had a 30% lower risk of developing heart disease compared to those who did not exercise regularly."""
        },
        {
            "instruction": """Label the paragraph as either Claim, Evidence, Corroboration, or Contextualization.
            paragraph: In recent years, there has been a significant shift in the automotive industry towards electric vehicles.
             This trend is influenced by growing concerns about climate change and the need for sustainable transportation solutions.
             Governments around the world have introduced policies and incentives to encourage the adoption of EVs, such as tax rebates
             and stricter emissions regulations. The rise in popularity is also driven by advancements in battery technology, which have improved
             the range and affordability of electric vehicles.""",
        },
        {
            "instruction": """Label the paragraph as either Claim, Evidence, Corroboration, or Contextualization.
            paragraph: Several studies support this claim. For example, a report by the U.S. Department of Energy found that
            electric vehicles cost about half as much to maintain and operate compared to gasoline vehicles over their lifetime.
            Additionally, a 2023 survey by Consumer Reports revealed that owners of electric vehicles spend 60% less on fuel and
            maintenance compared to owners of traditional vehicles. These findings are consistent with data from multiple sources that highlight
            the economic benefits of switching to electric vehicles.""",
        }

        ]

In [ ]:
with Pipeline(name="generate-dataset") as pipeline:

    load_data = LoadDataFromDicts(
        name="load_data",
        data=DATA,
        batch_size=1,
    )

    # Single TextGeneration step
    generate_response = TextGeneration(
        llm=InferenceEndpointsLLM(
            model_id="mistralai/Mistral-7B-Instruct-v0.3",
            tokenizer_id="mistralai/Mistral-7B-Instruct-v0.3",
            generation_kwargs={"max_new_tokens": 512, "temperature": 0.7},
        )
    )

    # evaluate_responses = UltraFeedback(
    #     aspect="overall-rating",
    #     llm=InferenceEndpointsLLM(
    #         model_id="meta-llama/Meta-Llama-3.1-70B-Instruct",
    #         tokenizer_id="meta-llama/Meta-Llama-3.1-70B-Instruct",
    #         generation_kwargs={"max_new_tokens": 512, "temperature": 0.7},
    #     )
    # )

  #  format_dpo = FormatTextGenerationDPO()

    # Connect the load_data step to the TextGeneration step
    load_data.connect(generate_response)

    # Connect TextGeneration to the next steps
    # generate_response.connect(evaluate_responses)
    # evaluate_responses.connect(format_dpo)


In [ ]:
distiset = pipeline.run(use_cache=False)

[09/21/24 11:37:55] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=775118;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=832699;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#696\696]8;;\
                             '/root/.cache/distilabel/pipelines/generate-dataset/f098727a9784f0f346a82f            
                             8d3de681627e5ba1d6/data'                                                              

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in     ]8;id=113148;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=660708;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#705\705]8;;\
                             stages:                                                                               
                              * Stage 0: ['load_data', 'text_generation_0']                                        

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to load... ]8;id=736898;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=312573;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#918\918]8;;\

[09/21/24 11:37:56] WARNING  ['distilabel.llm.mistralai/Mistral-7B-Instruct-v0.3'] Since ]8;id=984189;file:///usr/local/lib/python3.10/dist-packages/distilabel/llms/huggingface/inference_endpoints.py\inference_endpoints.py]8;;\:]8;id=12321;file:///usr/local/lib/python3.10/dist-packages/distilabel/llms/huggingface/inference_endpoints.py#190\190]8;;\
                             the                                                                                   
                             `base_url=https://api-inference.huggingface.co/models/mistr                           
                             alai/Mistral-7B-Instruct-v0.3` is available and either one                            
                             of `model_id` or `endpoint_name` is also provided, the                                
                             `base_url` will either be ignored or overwritten with the                             
                             one generated from either of those args, for serverless or                            
                             dedicated inference endpoints, respectively.                                          

[09/21/24 11:37:57] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 1/2                  ]8;id=390167;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=417040;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#950\950]8;;\
                              * 'load_data' replicas: 1/1                                                          
                              * 'text_generation_0' replicas: 0/1                                                  

...

/usr/local/lib/python3.10/dist-packages/distilabel/steps/tasks/base.py:183: UserWarning: `use_system_prompt` is set to `True`, but no `system_prompt` in input batch, so it will be ignored.
  return [self.format_input(input) for input in inputs]


                    INFO     ['distilabel.step.text_generation_0'] 📦 Processing batch 0 in     ]8;id=784901;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=376740;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#217\217]8;;\
                             'text_generation_0' (replica ID: 0)                                                   

[09/21/24 11:38:26] INFO     ['distilabel.step.text_generation_0'] 📨 Step 'text_generation_0'  ]8;id=400035;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=519315;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#274\274]8;;\
                             sending batch 0 to output queue                                                       

                    INFO     ['distilabel.step.text_generation_0'] 🏁 Finished running step     ]8;id=806655;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=852168;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#127\127]8;;\
                             'text_generation_0' (replica ID: 0)                                                   

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
distiset

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['instruction', 'generation', 'distilabel_metadata', 'model_name'],
            num_rows: 4
        })
    })
})

In [ ]:
distiset['default']['train'].to_pandas()

,instruction,generation,distilabel_metadata,model_name
0,"Label the paragraph as either Claim, Evidence,...",Claim: Regular exercise improves cardiovascula...,{'raw_output_text_generation_0': 'Claim: Regul...,mistralai/Mistral-7B-Instruct-v0.3
1,"Label the paragraph as either Claim, Evidence,...",Claim: Individuals who engage in at least 150 ...,{'raw_output_text_generation_0': 'Claim: Indiv...,mistralai/Mistral-7B-Instruct-v0.3
2,"Label the paragraph as either Claim, Evidence,...",Claim: There has been a significant shift in t...,{'raw_output_text_generation_0': 'Claim: There...,mistralai/Mistral-7B-Instruct-v0.3
3,"Label the paragraph as either Claim, Evidence,...","This paragraph can be labeled as Evidence, as ...",{'raw_output_text_generation_0': 'This paragra...,mistralai/Mistral-7B-Instruct-v0.3
